# To-do for first day of class

- Review Reddit code from Text Mining, implement it here. 
- Add more candidates, think about what fields we need
- Take all Twitter followers and their descriptions from candidates (this will take a fucking LONG time)
- Pull Twitter followers that don't follow the other candidates, or have candidate hashtags in descriptions
- take random sample. Think about random sample. e.g. Can we pull most active Tweeters? By Location?
- Look into Facebook API, try to implement it for candidates pages
- Make code only have to run once (Ideally every Sunday for the last week's data)
- Put data in GBQ
- Maybe query it into PostGres?
- Connect Tableau to GBQ/Postgres

In [66]:
import praw
import datetime
from datetime import timedelta
import nltk
from collections import defaultdict
from nltk.probability import FreqDist
from string import punctuation
from pprint import pprint
import random

from datetime import datetime
import json
from pprint import pprint
from twitter_functions import * # these are the functions John wrote.
# from twitter_config import *



ModuleNotFoundError: No module named 'twitter_config'

In [14]:
# Reddit credentials

reddit = praw.Reddit(client_id='QUxl6QyiQHgzJA',
                     client_secret='h64rlpNp6F6fg0VjvrF3knW3aAY',
                     user_agent='transcipt by u/jmcnea',
                     username='jmcnea',
                     password='b+3hKp/Pd.GA*2t')

In [67]:
# Twitter credentials

auth =  { "consumer_key": "qVwvZDIaSoqCc1jAxIEtPB3Tq",
         "consumer_secret": "M29LjXVBcZtDwt1iB9UcT3CR7Q20I0HhpQlchTFLCCy2wv7c9D",
         "access_key": "419210344-7E1HtetDiPyAsm1T1HU5GPMIsSd0wRqpjGaX3Vsk",
         "access_secret": "MNlEbxiKhuDs6GqXu2u3s6iT876Q4B4bSegUpxMJ2kwAr"
       }

api = initialize_twitter(auth)


In [72]:
candidates_handles = ['JoeBiden','BernieSanders', 'ewarren', 'PeteButtigieg', 'AndrewYang', 'realDonaldTrump']

for cand in candidates_handles:
    
    ofile_name = (datetime.today().strftime("%Y%m%d") + "_" + 
             cand + "_" + # Just use the first starting_user for the name if there are multiple
             "followers.txt")
    
    
    # We'll now go lookup the full information on your starting user(s). 
    starting_user_id = []

    # All records will be a dictionary with the twitter ID as the key and 
    # a UserRecord as the value. This is a named tuple I created. 
    all_records = lookup_users_from_handles(api, starting_user)

    # We need the IDs that we're getting followers from in a list. 
    for id in all_records : #access the keys, which are ids.
        starting_user_id.append(id)
        
        
    # How long is it going to take us to pull these followers?
    total_followers = 0
    for id, rec in all_records.items() :
        total_followers += rec.followers_count

    print("Ooh, {fol:,d} followers. A complete run with no limits run is ".format(fol=total_followers) + 
          "going to take {min:.2f} minutes ({hour:.2f} hours)".format(min=total_followers/5000,
                                                                      hour=total_followers/(60*5000)))

Start lookup_users_from_ids on 6 handles.
20191220-152135: looking up user records for 6 handles.
20191220-152135:  users pulled:  6
total failures: 0
Ooh, 88,159,237 followers. A complete run with no limits run is going to take 17631.85 minutes (293.86 hours)


In [ ]:
# Now let's pull all the followers of our starting_user
# the function I wrote allows you to cap the number of followers you pull
# and uses the ID to generate the query.
# 
# Note that this pull is subject to rate limiting. You can make 15 calls per
# 15 minute window and each can return 5000 users. 
followers_of_starting = gather_followers(api,
                                         starting_user_id,
                                         follower_limit=None) # Modify this limit if you need to. 
                                                              # Set it to "None" to get all   

# followers_of_starting will be a dictionary with the key being the id(s) in starting_user_id
# and the value is a list of all the followers.

Pulling followers for 939091
Number pulled: 5000
Number pulled: 10000
Number pulled: 15000
Number pulled: 20000
Number pulled: 25000
Number pulled: 30000
Number pulled: 35000
Number pulled: 40000
Number pulled: 45000
Number pulled: 50000
Number pulled: 55000
Number pulled: 60000
Number pulled: 65000
Number pulled: 70000
Number pulled: 75000


Rate limit reached. Sleeping for: 892


In [12]:
candidates_subreddits = ['sandersforpresident',
                        'elizabethwarren',
                        'pete_buttigieg',
                        'joebiden',
                        'yangforpresidenthq',
                        'the_donald']

In [38]:
### Reddit Comments

# Reddit records the date of comments being created in utc+8 time
# Haven't made sure this is the same for submissions...
# Next: break out after a week of time has passed

current_time = datetime.datetime.now(datetime.timezone.utc) + timedelta(hours = 8)
unix_timestamp = int(current_time.timestamp())
counter = 0
# It's been...
one_week = current_time - timedelta(days=7) + 
one_week_unix = int(one_week.timestamp())

date = ''.join(str(current_time).split(' ')[0].split('-'))

with open(f"C://Users//Jordan//Documents//Word Documents//AppliedData//reddit_pull_{date}.txt", 'w') as df:

    df.write('subreddit\tsubmission_id\tcomment_id\tcomment\tup_votes\tdown_votes\tusername\tdate\n')
    
    for candidate in candidates_subreddits:

        subreddit = reddit.subreddit(candidate)
        hot_python = subreddit.hot(limit=None)

        for submission in hot_python:

            if submission.created <= one_week_unix:
                break

            if not submission.stickied:

                comments = submission.comments.replace_more(limit=None)

                for comment in submission.comments.list():

                    if comment.created <= one_week_unix:
                        break

                    if comment.stickied:
                        continue

                    if comment.body == '[removed]' or comment == '[deleted]':
                        continue
                    

                    # this is writing fine, but the second line looks funky.
                    df.write(f'{candidate}\t{submission}\t{comment}\t'+
                    f'{comment.body.encode("utf-8",errors="ignore")}\t{comment.ups}\t{comment.downs}\t{comment.author}'+
                    f'\t{str(datetime.datetime.fromtimestamp(comment.created))}\n')



1576829236
1576797880
True
sandersforpresident
1576829236
1576798142
True
sandersforpresident
1576829236
1576803153
True
sandersforpresident
1576829236
1576800381
True
sandersforpresident
1576829236
1576797719
True
sandersforpresident
1576829236
1576798577
True
sandersforpresident
1576829236
1576800938
True
sandersforpresident
1576829236
1576803452
True
sandersforpresident
1576829236
1576805330
True
sandersforpresident
1576829236
1576808621
True
sandersforpresident
1576829236
1576794105
True
sandersforpresident
1576829236
1576807641
True
sandersforpresident
1576829236
1576813513
True
sandersforpresident
1576829236
1576802857
True
sandersforpresident
1576829236
1576815691
True
sandersforpresident
1576829236
1576815917
True
sandersforpresident
1576829236
1576816268
True
sandersforpresident
1576829236
1576818562
True
sandersforpresident
1576829236
1576812572
True
sandersforpresident
1576829236
1576812650
True
sandersforpresident
1576829236
1576813927
True
sandersforpresident
1576829236
15